In [1]:
import numpy as np
from skimage.measure import block_reduce

In [30]:
x = 4
y = 3
z = 2
arr = np.arange(x*y*z).reshape(x,y,z)
arr

array([[[ 0,  1],
        [ 2,  3],
        [ 4,  5]],

       [[ 6,  7],
        [ 8,  9],
        [10, 11]],

       [[12, 13],
        [14, 15],
        [16, 17]],

       [[18, 19],
        [20, 21],
        [22, 23]]])

## Block reduce

In [34]:
down_arr = block_reduce(arr, block_size=(2, 2, 2), func=np.mean)
down_arr, down_arr.shape

(array([[[ 4.5 ],
         [ 3.75]],
 
        [[16.5 ],
         [ 9.75]]]),
 (2, 2, 1))

In [32]:
block_size = (2, 2, 2)
downsampled_block_indices = (1, 1, 0)

block_indices = []
# initialize block_indices - [(0,0,0), ...., (block_size[0]-1, block_size[1]-1, block_size[2]-1)]
for i in range(block_size[0]):
    for j in range(block_size[1]):
        for k in range(block_size[2]):
            block_indices.append((i, j, k))

# get the indices of the block to be downsampled in the original array
block_indices = np.array(block_indices)
block_indices += np.array(downsampled_block_indices) * np.array(block_size)

# validate that the block indices are within the original array
# else remove the indices that are out of bounds
block_indices = block_indices[block_indices[:, 0] < arr.shape[0]]
block_indices = block_indices[block_indices[:, 1] < arr.shape[1]]
block_indices = block_indices[block_indices[:, 2] < arr.shape[2]]

block_indices

array([[2, 2, 0],
       [2, 2, 1],
       [3, 2, 0],
       [3, 2, 1]])

In [33]:
print(arr[block_indices[:, 0], block_indices[:, 1], block_indices[:, 2]])
print(np.mean(arr[block_indices[:, 0], block_indices[:, 1], block_indices[:, 2]]))
print(down_arr[downsampled_block_indices])

[16 17 22 23]
19.5
9.75


In [50]:
def downsample_indices(arr, block_size, input_coords, downsampled_image):
    downsampled_shape = downsampled_image.shape
    indices = []
    for i in range(block_size[0] * input_coords[0], block_size[0] * (input_coords[0] + 1)):
        for j in range(block_size[1] * input_coords[1], block_size[1] * (input_coords[1] + 1)):
            for k in range(block_size[2] * input_coords[2], block_size[2] * (input_coords[2] + 1)):
                if i < arr.shape[0] and j < arr.shape[1] and k < arr.shape[2]:
                    indices.append((i, j, k))
    return np.array(indices)

In [55]:
ia = downsample_indices(arr, (2,2,2), (1, 1, 0), down_arr)
ia
# print(arr[ia[:, 0], ia[:, 1], ia[:, 2]])
# print(np.sum(arr[ia[:, 0], ia[:, 1], ia[:, 2]])/(block_size[0]*block_size[1]*block_size[2]))
# print(down_arr[1, 1, 0])

array([[2, 2, 0],
       [2, 2, 1],
       [3, 2, 0],
       [3, 2, 1]])

In [56]:
def downsample_indices(arr, block_size, input_coords):
    block_row_indices = np.arange(block_size[0] * input_coords[0], block_size[0] * (input_coords[0] + 1))
    block_col_indices = np.arange(block_size[1] * input_coords[1], block_size[1] * (input_coords[1] + 1))
    block_depth_indices = np.arange(block_size[2] * input_coords[2], block_size[2] * (input_coords[2] + 1))
    
    block_row_indices = block_row_indices[block_row_indices < arr.shape[0]]
    block_col_indices = block_col_indices[block_col_indices < arr.shape[1]]
    block_depth_indices = block_depth_indices[block_depth_indices < arr.shape[2]]
    
    row_grid, col_grid, depth_grid = np.meshgrid(block_row_indices, block_col_indices, block_depth_indices)
    
    return np.column_stack((row_grid.ravel(), col_grid.ravel(), depth_grid.ravel()))


In [59]:
downsample_indices(arr, (2,2,2), (1, 1, 0))

array([[2, 2, 0],
       [2, 2, 1],
       [3, 2, 0],
       [3, 2, 1]])